In [2]:
import torch
import torch.nn as nn
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import os

from tqdm import tqdm
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LinearRegression
from pprint import pprint
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, roc_auc_score, classification_report
from sklearn.linear_model import Ridge
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
from dateutil.parser import parse
import os
import random

plt.style.use('fivethirtyeight')

In [1]:
!pip install catboost
!pip install xgboost
!pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 25.2 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
PATH = "/content/drive/My Drive/Data/train_data/"
target = pd.read_csv("/content/drive/My Drive/Data/train_target.csv")
target_list = target['flag'].to_numpy()
target = 0

In [5]:
tmp = [17, 18, 19, 20, 21, 25, 26, 27, 57, 58]
cat_features = []
for i in range(60):
    for j in tmp:
        cat_features.append(i*59 + j)
cat_features = np.array(cat_features, int)

In [6]:
start = 0
end = 250000
zeros_cnt = 10000000

ones = []
for i in range(12):
    # Парсинг данных
    if (i == 12):
        start = 0
        end = 250000
    filename = "train_data_" + str(i % 12) + ".pq"
    df = pq.read_table(source=PATH + filename).to_pandas()
    columns = len(df.columns)
    X_t = []
    # temp_ids = sorted(set(df['id'].values))
    # ids = []
    # for id in temp_ids:
    #     if target_list[id] == 0:
    #         ids.append(id)
    # ids = random.sample(ids, zeros_cnt)
    ids = sorted(set(df['id'].values))
    max_size = 60
    y_t = target_list[start:end]
    start += 250000
    end += 250000
    cur_zeros_cnt = 0
    index = -1
    for id in ids:
        index += 1
        if y_t[index] == 0 and cur_zeros_cnt >= zeros_cnt:
            continue
        if y_t[index] == 0:
            cur_zeros_cnt += 1
            temp = df[df['id'] == id]
            temp = temp.drop(columns=['id', 'rn'])
            temp_np = np.concatenate((temp.to_numpy().reshape((columns - 2) * len(temp)), np.zeros((columns - 2) * (max_size - len(temp)))))
            X_t.append(temp_np)
        else:
            temp = df[df['id'] == id]
            temp = temp.drop(columns=['id', 'rn'])
            temp_np = np.concatenate((temp.to_numpy().reshape((columns - 2) * len(temp)), np.zeros((columns - 2) * (max_size - len(temp)))))
            ones.append(temp_np)

    df = None
    X_t = np.array(X_t, int)
    X_t = pd.DataFrame(data=X_t)
    X_t.to_csv("./Data/X_zeros_" + str(i) + ".csv", index=False)

    # if zeros is None:
    #     zeros = X_t
    # else:
    #     zeros = pd.concat([zeros, X_t], ignore_index=True)
    # X_t.to_csv("./MyData/X_train_" + str(i) + ".csv", index=False)
    # y_t.to_csv("./MyData/y_train_" + str(i) + ".csv", index=False)

    # Разбивка на train и test данные
    # X_train, X_test, y_train, y_test = train_test_split(X_t, y_t, train_size=0.9, random_state=42)
    '''
    # Если первая итерация то обучаем с нуля, иначе продолжаем обучать старую модель
    if i == 0:
        print("Iteration " + str(i))
        CatBoostModel.fit(X_t, y_t,
            cat_features=cat_features,
            verbose=False
        )
    else:
        CatBoostModel.fit(X_t, y_t,
            init_model=CatBoostModel,
            cat_features=cat_features,
            verbose=False
        )
    '''
    # X_train = None
    # X_test = None
    # y_test = None
    # y_train = None
    X_t = None
    y_t = None
ones = np.array(ones, int)
ones = pd.DataFrame(data=ones)
ones.to_csv("./Data/ones.csv", index=False)

In [ ]:
# ones = pd.read_csv("./ones.csv")
# l_o = len(ones)
# ones = None
# ones = pd.DataFrame(data=np.ones(l_o), dtype=int)
# ones.to_csv("./y_ones.csv", index=False)
# ones = None
# zeros = pd.read_csv("./zeros.csv")
# l_z = len(zeros)
# zeros = None
# zeros = pd.DataFrame(data=np.zeros(l_z), dtype=int)
# zeros.to_csv("./y_zeros.csv", index=False)
# zeros = None

In [ ]:
# ones = pd.read_csv("./ones.csv")
# zeros = pd.read_csv("./zeros.csv")
# X_t = pd.concat([ones, zeros], ignore_index=True)
# ones = None
# zeros = None

# ones = pd.read_csv("./y_ones.csv")
# zeros = pd.read_csv("./y_zeros.csv")
# y_t = pd.concat([ones, zeros], ignore_index=True)
# ones = None
# zeros = None

In [7]:
X_t = None
dfs = []
zeros_num = 0
for i in range(12):
  df = pd.read_csv("./Data/X_zeros_" + str(i) + ".csv")
  df_percent = df.sample(frac=1)
  df = None
  zeros_num += len(df_percent)
  dfs.append(df_percent)
  df_percent = None
  print(i)
ones = pd.read_csv("./Data/ones.csv")
cnt = 3
for i in range(cnt):
  dfs.append(ones)
X_t = pd.concat(dfs, ignore_index=True)
y_t = pd.concat([pd.DataFrame(data=np.zeros(zeros_num)), pd.DataFrame(data=np.ones(cnt * len(ones)))])
ones = None
dfs = None

0
1
2
3
4
5
6
7
8
9
10
11


In [ ]:
# for mod in range(9, 10):
#   CatBoostModel = cb.CatBoostClassifier(
#     task_type="CPU",
#     eval_metric='AUC',
#     iterations=1500,
#     learning_rate=0.1,
#     random_seed=63,
#     custom_loss=['AUC'],
#     early_stopping_rounds=50
#   )
#   X_t = None
#   dfs = []
#   zeros_num = 0
#   for i in range(12):
#     df = pd.read_csv("./Data/X_zeros_" + str(i) + ".csv")
#     df_percent = df.sample(frac=0.4)
#     df = None
#     zeros_num += len(df_percent)
#     dfs.append(df_percent)
#     df_percent = None
#   ones = pd.read_csv("./Data/ones.csv")
#   df_percent = ones.sample(frac=0.8)
#   dfs.append(df_percent)
#   X_t = pd.concat(dfs, ignore_index=True)
#   y_t = pd.concat([pd.DataFrame(data=np.zeros(zeros_num)), pd.DataFrame(data=np.ones(len(df_percent)))], ignore_index=True)
#   df_percent = None
#   ones = None
#   dfs = None

#   X_train, X_test, y_train, y_test = train_test_split(X_t, y_t, train_size=0.99, random_state=42)
#   X_t = None
#   y_t = None

#   CatBoostModel.fit(
#     X_train, y_train,
#     cat_features=cat_features,
#     verbose=False
#   )

#   models.append(CatBoostModel)
#   CatBoostModel.save_model('./Models/Model_' + str(mod) + '.json')
#   CatBoostModel.save_model('./Models/Model_' + str(mod) + '.bin')
#   print('Model ' + str(mod) + ' done')


Model 9 done


In [ ]:
# predictions1 = []
# predictions2 = []
# for i in range(0, 2):
#     X_t = pd.read_csv("X_test_" + str(i) + ".csv")
#     for mod in range(10):
#       probability_predictions = models[mod].predict_proba(X_t)[:,1]
#       if i == 0:
#           predictions1.append(probability_predictions)
#       else:
#           predictions2.append(probability_predictions)
#       probability_predictions = None
#     X_t = None


In [ ]:
# pred1 = np.zeros(len(predictions1[0]), float)
# pred2 = np.zeros(len(predictions2[0]), float)
# for mod in range(10):
#   for i in range(len(predictions1[mod])):
#     pred1[i] += predictions1[mod][i]

# for mod in range(10):
#   for i in range(len(predictions2[mod])):
#     pred2[i] += predictions2[mod][i]

# pred1 = pred1 / 10
# pred2 = pred2 / 10

In [ ]:
# predictions = np.concatenate((pred1, pred2))
# ans = []
# for i in range(3000000, 3500000):
#     ans.append([i, predictions[i-3000000]])
# df = pd.DataFrame(ans)
# df.columns = ['id', 'score']
# df.to_csv('./res.csv', index=False)

In [8]:
CatBoostModel = cb.CatBoostClassifier(
    task_type="CPU",
    eval_metric='AUC',
    iterations=5000,
    depth=2,
    learning_rate=0.1,
    random_seed=63,
    custom_loss=['AUC'],
    early_stopping_rounds=50
)

grid = {
    'iterations': [900, 1000, 1100, 1200, 1300],
    'learning_rate': [0.075, 0.1, 0.15],
    'depth': [2, 4, 6, 8]
}

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_t, y_t, train_size=0.99, random_state=42)
X_t = None
y_t = None

In [10]:
CatBoostModel.fit(
    X_train, y_train,
    cat_features=cat_features,
)

0:	total: 383ms	remaining: 31m 54s
1:	total: 748ms	remaining: 31m 9s
2:	total: 1.07s	remaining: 29m 43s
3:	total: 1.35s	remaining: 28m 2s
4:	total: 1.68s	remaining: 27m 54s
5:	total: 1.95s	remaining: 27m 2s
6:	total: 2.27s	remaining: 26m 59s
7:	total: 2.6s	remaining: 27m 5s
8:	total: 2.93s	remaining: 27m 7s
9:	total: 3.26s	remaining: 27m 6s
10:	total: 3.58s	remaining: 27m 5s
11:	total: 3.92s	remaining: 27m 11s
12:	total: 4.24s	remaining: 27m 8s
13:	total: 4.57s	remaining: 27m 6s
14:	total: 4.89s	remaining: 27m 4s
15:	total: 5.21s	remaining: 27m 3s
16:	total: 5.53s	remaining: 27m 1s
17:	total: 5.85s	remaining: 26m 58s
18:	total: 6.17s	remaining: 26m 56s
19:	total: 6.48s	remaining: 26m 53s
20:	total: 6.81s	remaining: 26m 53s
21:	total: 7.12s	remaining: 26m 50s
22:	total: 7.45s	remaining: 26m 51s
23:	total: 7.77s	remaining: 26m 51s
24:	total: 8.09s	remaining: 26m 49s
25:	total: 8.41s	remaining: 26m 48s
26:	total: 8.73s	remaining: 26m 47s
27:	total: 9.04s	remaining: 26m 45s
28:	total: 9.37

In [11]:
CatBoostModel.save_model('./CB.json')

In [ ]:
predictions1 = []
predictions2 = []
for i in range(0, 2):
    # Парсинг данных
    probability_predictions = None
    filename = "test_data_" + str(i) + ".pq"
    df = pq.read_table(source="/content/drive/My Drive/Data/test_data/" + filename).to_pandas()
    columns = len(df.columns)
    X_t = []
    ids = sorted(set(df['id'].values))
    max_size = 60
    for id in ids:
        temp = df[df['id'] == id]
        temp = temp.drop(columns=['id', 'rn'])
        temp_np = np.concatenate((temp.to_numpy().reshape((columns - 2) * len(temp)), np.zeros((columns - 2) * (max_size - len(temp)))))
        X_t.append(temp_np)

    df = None
    X_t = np.array(X_t, int)
    X_t = pd.DataFrame(data=X_t)
    # X_t.to_csv("./X_test_" + str(i) + ".csv", index=False)
    # X_t = pd.read_csv("X_test_" + str(i) + ".csv")
    probability_predictions = CatBoostModel.predict_proba(X_t)
    if i == 0:
        predictions1 = probability_predictions
    else:
        predictions2 = probability_predictions
    probability_predictions = None
    X_t = None

In [ ]:
predictions = np.concatenate((predictions1, predictions2))
ans = []
for i in range(3000000, 3500000):
    ans.append([i, predictions[i-3000000,1]])
df = pd.DataFrame(ans)
df.columns = ['id', 'score']
df.to_csv('./CB.csv', index=False)

In [ ]:
import xgboost as xgb

XGBModel = xgb.XGBClassifier(
 learning_rate=0.1,
 n_estimators=1700,
 objective= 'binary:logistic',
 seed=27)

XGBModel.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1700, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
XGBModel.save_model("XGB5.json")

In [ ]:
predictions1 = []
predictions2 = []
for i in range(0, 2):
    '''# Парсинг данных
    probability_predictions = None
    filename = "test_data_" + str(i) + ".pq"
    df = pq.read_table(source="/content/drive/My Drive/Data/test_data/" + filename).to_pandas()
    columns = len(df.columns)
    X_t = []
    ids = sorted(set(df['id'].values))
    max_size = 60
    for id in ids:
        temp = df[df['id'] == id]
        temp = temp.drop(columns=['id', 'rn'])
        temp_np = np.concatenate((temp.to_numpy().reshape((columns - 2) * len(temp)), np.zeros((columns - 2) * (max_size - len(temp)))))
        X_t.append(temp_np)

    df = None
    X_t = np.array(X_t, int)
    X_t = pd.DataFrame(data=X_t)
    X_t.to_csv("./X_test_" + str(i) + ".csv", index=False)'''
    X_t = pd.read_csv("X_test_" + str(i) + ".csv")
    probability_predictions = XGBModel.predict_proba(X_t)
    if i == 0:
        predictions1 = probability_predictions
    else:
        predictions2 = probability_predictions
    probability_predictions = None
    X_t = None

In [ ]:
predictions = np.concatenate((predictions1, predictions2))
ans = []
for i in range(3000000, 3500000):
    ans.append([i, predictions[i-3000000,1]])
df = pd.DataFrame(ans)
df.columns = ['id', 'score']
df.to_csv('./XGB5.csv', index=False)

In [ ]:
start = 0
end = 250000
zeros_cnt = 10000000

ones = []
for i in range(11, 12):
    # Парсинг данных
    if (i == 12):
        start = 0
        end = 250000
    filename = "train_data_" + str(i % 12) + ".pq"
    df = pq.read_table(source=PATH + filename).to_pandas()
    columns = len(df.columns)
    X_t = []
    # temp_ids = sorted(set(df['id'].values))
    # ids = []
    # for id in temp_ids:
    #     if target_list[id] == 0:
    #         ids.append(id)
    # ids = random.sample(ids, zeros_cnt)
    ids = sorted(set(df['id'].values))
    max_size = 60
    y_t = target_list[start:end]
    start += 250000
    end += 250000
    cur_zeros_cnt = 0
    index = -1
    for id in ids:
        index += 1
        if y_t[index] == 0 and cur_zeros_cnt >= zeros_cnt:
            continue
        temp = df[df['id'] == id]
        temp = temp.drop(columns=['id', 'rn'])
        temp_np = np.concatenate((temp.to_numpy().reshape((columns - 2) * len(temp)), np.zeros((columns - 2) * (max_size - len(temp)))))
        X_t.append(temp_np)

    df = None
    X_t = np.array(X_t, int)
    X_t = pd.DataFrame(data=X_t)
    X_t.to_csv("./Val/X_val.csv", index=False)

    # if zeros is None:
    #     zeros = X_t
    # else:
    #     zeros = pd.concat([zeros, X_t], ignore_index=True)
    # X_t.to_csv("./MyData/X_train_" + str(i) + ".csv", index=False)
    # y_t.to_csv("./MyData/y_train_" + str(i) + ".csv", index=False)

    # Разбивка на train и test данные
    # X_train, X_test, y_train, y_test = train_test_split(X_t, y_t, train_size=0.9, random_state=42)
    '''
    # Если первая итерация то обучаем с нуля, иначе продолжаем обучать старую модель
    if i == 0:
        print("Iteration " + str(i))
        CatBoostModel.fit(X_t, y_t,
            cat_features=cat_features,
            verbose=False
        )
    else:
        CatBoostModel.fit(X_t, y_t,
            init_model=CatBoostModel,
            cat_features=cat_features,
            verbose=False
        )
    '''
    # X_train = None
    # X_test = None
    # y_test = None
    # y_train = None
    X_t = None
    y_t = None

In [ ]:
CatBoostModel = cb.CatBoostClassifier(
    task_type="CPU",
    eval_metric='AUC',
    iterations=1700,
    learning_rate=0.1,
    random_seed=63,
    custom_loss=['AUC'],
    early_stopping_rounds=50
)
CatBoostModel.load_model('CB06.json')

In [ ]:
XGBModel = xgb.XGBClassifier(
  learning_rate=0.1,
  n_estimators=1700,
  objective= 'binary:logistic',
  seed=27
)
XGBModel.load_model("XGB06.json")

In [ ]:
X_t = pd.read_csv("./Val/X_val.csv")

In [ ]:
predictions1 = CatBoostModel.predict_proba(X_t)
predictions2 = XGBModel.predict_proba(X_t)
ans = []
for i in range(2750000,3000000):
    ans.append([i, predictions1[i-2750000,1]])
df = pd.DataFrame(ans)
df.columns = ['id', 'score']
df.to_csv('./TrainData/CB5.csv', index=False)
ans = []
for i in range(2750000,3000000):
    ans.append([i, predictions2[i-2750000,1]])
df = pd.DataFrame(ans)
df.columns = ['id', 'score']
df.to_csv('./TrainData/XGB5.csv', index=False)